
# Working with iMOD5 models in MODFLOW 6

This example shows how to work with iMOD5 models in MODFLOW 6. It demonstrates
how to convert an iMOD5 model to a MODFLOW 6 model using the `imod` package. The
example fetches an iMOD5 model, converts it to a MODFLOW 6 model, regrids it to
an unstructured grid, and compares differences in output between the structured
and unstructured grids.


In [ ]:
import imod

## Fetching an iMOD5 model

Let's start by fetching the example data
from the `imod.data` module. This will download a project file and
accompanying data files to a temporary directory.



In [ ]:
tmpdir = imod.util.temporary_directory()

prj_dir = tmpdir / "prj"
prj_dir.mkdir(exist_ok=True, parents=True)

model_dir = imod.data.fetch_imod5_model(prj_dir)

Let's view the model directory. We can use ``.glob("*")`` to view all
directory contents. The directory contains the project file and accompanying
model contents.



In [ ]:
from pprint import pprint

imod_dir_contents = list(model_dir.glob("*"))
pprint(imod_dir_contents)

The directory contains a project file and a database folder.
This database contains all the IDF, IPF, and GEN files that make up the
spatial model input.

Let's look at the projectfile contents. Read the projectfile as follows:



In [ ]:
prj_path = model_dir / "iMOD5_model.prj"
prj_content = imod.prj.read_projectfile(prj_path)
pprint(prj_content)

This contains all the projectfile contents in a dictionary, which is quite a
lot of information. This is too much to go through in detail. We can also open
all data that the projectfile points to, using the
:func:`imod.prj.open_projectfile_data` function.



In [ ]:
imod5_data, period_data = imod.prj.open_projectfile_data(prj_path)
imod5_data

This groups all data per package in the projectfile into a dictionary with
DataArrays per variable.



In [ ]:
imod5_data["riv-1"]["stage"]

Let's plot the stage data of the first river package.



In [ ]:
imod5_data["riv-1"]["stage"].isel(layer=0, drop=True).plot.imshow()

## Converting iMOD5 model to MODFLOW 6

This is nice enough, but we want to convert this iMOD5 model to a MODFLOW 6
model. We can do this using
:meth:`imod.mf6.Modflow6Simulation.from_imod5_data` method. Next to the iMOD5
data and period data, we also need to provide the times. These will be used to
resample the asynchronous well timeseries data to these times. For instance,
well 1 in the iMOD5 database can have rates specified on a daily basis,
whereas well 2 is specified on a few days in the year. Say the user wants to
run a model on a monthly basis, this will require resampling these rate
timeseries to make them consistent with the simulation timesteps. Let's
therefore first create a list of times which will be the simulation's
timesteps, we can use pandas for this. "MS" stands for "month start", meaning
the first day of each month.



In [ ]:
import pandas as pd

times = pd.date_range(start="2020-01-01", periods=10, freq="MS")
times

Now that we have a list of times, we can import the iMOD5 data into a MODFLOW
6 simulation. This might require some time, as it will convert all the iMOD5
data to be compatible with MODFLOW . For example, the river systems with
infiltration factors are transformed into a separate
Drain and River package (if necessary) to get the same behavior as iMOD5's
infiltration factors.



In [ ]:
mf6_sim = imod.mf6.Modflow6Simulation.from_imod5_data(imod5_data, period_data, times)
mf6_sim

## Improving the solver settings

At the moment the MODFLOW 6 simulation has quite loose solver settings. Most
notably, the ``inner_dvclose`` is set to 0.01, which means that the solver
allows a numerical error of 1 cm in the head values. This is quite loose.



In [ ]:
mf6_sim["ims"]

This is because by default an iMOD5 model is imported with a
SolutionPresetModerate, which is quite loose. Let's set a stricter solver
setting preset, by setting it to SolutionPresetSimple. This has a
``inner_dvclose`` of 0.001, which allows a numerical error of 1 mm in the head
values.



In [ ]:
mf6_sim["ims"] = imod.mf6.SolutionPresetSimple(["imported_model"])
mf6_sim["ims"]

## A note on performance

By default, the iMOD5 model rasters will not be directly loaded into memory,
but instead will be lazily loaded. Read more about this in the
:doc:`06-lazy-evaluation` documentation.

By default the data is chunked per raster file, which is a chunk per layer,
per timestep. Usually this is not optimal, as this creates many small chunks.

## Writing the structured model: in fits and starts

Let's try to write this simulation. **spoiler alert**: this will fail, because
we still have to configure some packages.



In [ ]:
mf6_dir = tmpdir / "mf6_structured"

# Ignore this "with" statement, it is to catch the error and render the
# documentation without error.
with imod.util.print_if_error(ValueError):
    mf6_sim.write(mf6_dir)  # Attention: this will fail!

We are still missing output control, as the projectfile does not contain this
information. For this example, we'll only save the last head of each stress period.



In [ ]:
gwf_model = mf6_sim["imported_model"]
gwf_model["oc"] = imod.mf6.OutputControl(
    save_head="last",
)

from imod.schemata import ValidationError

with imod.util.print_if_error(ValidationError):
    mf6_sim.write(mf6_dir)  # Attention: this will fail!

Argh! The simulation still fails to write. In general, iMOD Python is a lot
stricter with writing model data than iMOD5. iMOD Python forces users to
conciously clean up their models, whereas iMOD5 cleaned data under the hood.
The error message states that the nodata is not aligned with idomain. This
means there are k values and ic values specified at inactive locations, or
vice versa. Let's try if masking the data works. This will remove all inactive
locations (``idomain > 0``) from the data.



In [ ]:
idomain = gwf_model["dis"]["idomain"]
mf6_sim.mask_all_models(idomain)

mf6_sim.write(mf6_dir)

## Running the structured model

Let's run the simulation and open the head data.



In [ ]:
mf6_sim.run()
head_structured = mf6_sim.open_head()
# Plot the head of the last stress period at layer 5.
head_structured.isel(time=-1).sel(layer=5).plot.imshow()

## Regridding the structured model to an unstructured grid

Now that we have a MODFLOW 6 simulation, we can regrid it to an unstructured
grid. Let's first load a triangular grid.



In [ ]:
triangular_grid = imod.data.lhm_clip_triangular_grid()
triangular_grid.plot()

That looks more exciting than the rectangular grid we had before. You can see
there is refinement around some of the streams and especially around
horizontal flow barriers. We haven't looked at horizontal flow barriers yet,
so let's plot them on top of the triangular mesh.



In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
triangular_grid.plot(ax=ax, color="lightgrey", edgecolor="black")
gwf_model["hfb-25"].line_data.plot(ax=ax, color="blue", linewidth=2)
gwf_model["hfb-26"].line_data.plot(ax=ax, color="blue", linewidth=2)

However, this grid is triangular, which has the disadvantage that the connections
between cell centers are not orthogonal to the cell edges, which can lead to
mass balance errors. xugrid has a method to convert this triangular grid
to a Voronoi grid, which has orthogonal connections between cell centers and
edges.



In [ ]:
voronoi_grid = triangular_grid.tesselate_centroidal_voronoi()
voronoi_grid.plot()

# iMOD Python regridding functionality requires a UgridDataArray instead of a
# Ugrid2d, so we create a UgridDataArray with the voronoi grid.
from imod.util import ones_like_ugrid

voronoi_uda = ones_like_ugrid(voronoi_grid)

Now that we have a Voronoi grid, we can regrid the MODFLOW 6 simulation to this
grid.



In [ ]:
mf6_unstructured = mf6_sim.regrid_like("unstructured_example", voronoi_uda)
mf6_unstructured

Let's take a gander at how the river data is regridded.



In [ ]:
mf6_unstructured["imported_model"]["riv-1riv"]["stage"].isel(layer=0).ugrid.plot()

## Writing the unstructured model: in more fits and starts

Let's try to write this to a temporary directory. **Spoiler alert**: Like
before, this will fail.



In [ ]:
mf6_dir = tmpdir / "mf6_unstructured"

# Ignore this "with" statement, it is to catch the error and render the
# documentation without error.
with imod.util.print_if_error(ValidationError):
    mf6_unstructured.write(mf6_dir)  # Attention: this will fail!

The error message states that the iMOD5 model has a river package that has its
river bottom elevation below the model bottom. The averaging when regridding
can cause this: The model bottom has a continuous surface, whereas the rivers
usually are located in a local valley. Upscaling both with a mean causes the
river bottom elevation to have the tendency to be lower than the model bottom.
We therefore need to reallocate the river data to the new model layer
schematization.



In [ ]:
gwf_unstructured = mf6_unstructured["imported_model"]
dis = gwf_unstructured["dis"]
npf = gwf_unstructured["npf"]

gwf_unstructured["riv-1riv"] = gwf_unstructured["riv-1riv"].reallocate(dis, npf)
gwf_unstructured["riv-2riv"] = gwf_unstructured["riv-2riv"].reallocate(dis, npf)
gwf_unstructured["riv-1drn"] = gwf_unstructured["riv-1drn"].reallocate(dis, npf)
gwf_unstructured["riv-2drn"] = gwf_unstructured["riv-2drn"].reallocate(dis, npf)
gwf_unstructured["riv-1riv"].cleanup(dis)
gwf_unstructured["riv-2riv"].cleanup(dis)
gwf_unstructured["riv-1drn"].cleanup(dis)
gwf_unstructured["riv-2drn"].cleanup(dis)

We'll now be able to finally write the unstructured model.



In [ ]:
mf6_unstructured.write(mf6_dir)

## Running the unstructured model

Let's run the unstructured model and open the head data.



In [ ]:
mf6_unstructured.run()
head_unstructured = mf6_unstructured.open_head()
# Plot the head of the last stress period at layer 5.
head_unstructured.isel(time=-1).sel(layer=5).ugrid.plot()

## Comparing differences in output

In this section, we will compare the output of the structured and unstructured
models. We fill first plot the structured and unstructured results side by
side. Next, we will regrid the structured head data to the unstructured grid
and compare the differences in output. This will show how the regridding
affects the output.

### Side-by-side comparison

Let's first plot the structured and unstructured head data side by side.



In [ ]:
head_structured_end_l5 = head_structured.isel(time=-1).sel(layer=5)
head_unstructured_end_l5 = head_unstructured.isel(time=-1).sel(layer=5)

import matplotlib.pyplot as plt

# Get the data range from both datasets
vmin = min(head_structured_end_l5.min().values, head_unstructured_end_l5.min().values)
vmax = max(head_structured_end_l5.max().values, head_unstructured_end_l5.max().values)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 6), width_ratios=(1, 1, 0.1))
head_structured_end_l5.plot.imshow(ax=axes[0], add_colorbar=False, vmin=vmin, vmax=vmax)
axes[0].set_title(f"Structured grid (ncells = {head_structured_end_l5.size})")
head_unstructured_end_l5.ugrid.plot(ax=axes[1], cbar_ax=axes[2], vmin=vmin, vmax=vmax)
axes[1].set_title(f"Unstructured grid (ncells = {head_unstructured_end_l5.size})")
axes[1].get_yaxis().set_visible(False)

# Get the y-limits from the first axis and apply to both
ylim = axes[0].get_ylim()
axes[1].set_ylim(ylim)

### Computing differences

Next, we will compute the differences between the structured and unstructured
head data. This will show how the regridding affects the output in more
detail. For that we first need to upscale the structured head data to the
unstructured grid. This is done using the [OverlapRegridder from the xugrid
package](https://deltares.github.io/xugrid/examples/regridder_overview.html#overlapregridder),



In [ ]:
import xugrid as xu

regridder = xu.OverlapRegridder(head_structured, head_unstructured.ugrid.grid)
head_structured_upscaled = regridder.regrid(head_structured)

Compute the difference between the upscaled structured head and the
unstructured head. A zero difference means the regridding didn't result in any
differences. We can see around the western fault that the regridding caused
differences.



In [ ]:
diff = (head_structured_upscaled - head_unstructured).isel(time=-1).compute()
diff.mean(dim="layer").ugrid.plot()

Let's also plot the standard deviation of the difference. This shows that
variations in difference are also mostly around the western fault.



In [ ]:
diff.std(dim="layer").ugrid.plot()

### Differences in detail

This is a good example of how regridding can lead to differences in output:
The line representing the fault has to be snapped to the cell edges. This is
strongly grid dependent. And can lead to local differences in output. Let's
visualize how faults are snapped to the grid edges in the structured and
unstructured grid.



In [ ]:
structured_snapped = gwf_model["hfb-5"].snap_to_grid(gwf_model["dis"])
unstructured_snapped = gwf_unstructured["hfb-5"].snap_to_grid(gwf_unstructured["dis"])

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
structured_snapped["resistance"].ugrid.plot(add_colorbar=False, ax=ax)
unstructured_snapped["resistance"].ugrid.plot(add_colorbar=False, ax=ax)
diff.mean(dim="layer").ugrid.plot(ax=ax)
ax.set_xlim(197500, 198500)
ax.set_ylim(361000, 363000)

EXERCISE: Download this file as a script or Jupyter notebook, remove all HFB
packages and re-run the example. Investigate if differences are still as large
as they were. You can remove the HFB packages by using the ``pop`` method on
the groundwater flow model. There are 26 HFB packages in the model, so you
can use a for loop to remove them all. The HFB packages are named
"hfb-1", "hfb-2", ..., "hfb-26".

